# Distributed Inference and Serving

Common distributed inference strategies according to your hardware resources:
- Single GPU
    - No distributed inference.
- Single-node, multi-GPU
    - tensor parallel: model is too large to fit in one GPU but fit in multiple GPUs within a single node.
        - tensor parallel size = number of GPUs
- Multi-node, multi-GPU
    - tensor parallel + pipeline parallel
        - tensor parallel size = number of GPUs per node
        - pipeline parallel size = number of nodes

## Inference memory calculation

$$ M = \frac{P * 4}{32/Q} * 1.2 (GB) $$

- $M$: GPU memory
- $P$: parameters (in Billions, 7B is 7)
- $4$: 4 bytes for each parameter
- $32$: 1 byte is 8 bits, so 4 bytes is 32 bits
- $Q$: quantization bits (e.g., 16 bits, 8 bits, 4 bits)
- $1.2$: 20% overhead

Memory for loading 70B model at 16 bit precision is

$$ \frac{70 * 4}{32/16} * 1.2 = 168GB $$

Tool: [Can you run it](https://huggingface.co/spaces/Vokturz/can-it-run-llm)

## Use vLLM to do inference Llama-3.3-70B-Instruct

### Distributed inference

We have a `Standard_NC80adis_H100_v5` AML VM, which has 2 NVIDIA H100 GPUs with 94 GiB vRAM each (188 GiB together). 

As calculated above, doing inference for a 70B model at FP16 precision needs 168GB vRAM. With the VM at hand, we can use the combined 188GiB vRAM of both GPUs to cover the 168GiB memory needs. With vLLM we use tensor parallel by specifying vLLM serve parameter `--tensor-parallel-size 2`.

```bash
$ vllm serve unsloth/Llama-3.3-70B-Instruct \
$  --tensor-parallel-size 2
```

At start vLLM will log all of its args. The ones that weren't specified manually will have their default values assigned.

```bash
INFO 12-08 09:28:05 api_server.py:585] vLLM API server version 0.6.4.post1
INFO 12-08 09:28:05 api_server.py:586] args: Namespace(subparser='serve', model_tag='unsloth/Llama-3.3-70B-Instruct', config='', host=None, port=8000, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, lora_modules=None, prompt_adapters=None, chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], return_tokens_as_token_ids=False, disable_frontend_multiprocessing=False, enable_auto_tool_choice=False, tool_call_parser=None, tool_parser_plugin='', model='unsloth/Llama-3.3-70B-Instruct', task='auto', tokenizer=None, skip_tokenizer_init=False, revision=None, code_revision=None, tokenizer_revision=None, tokenizer_mode='auto', chat_template_text_format='string', trust_remote_code=False, allowed_local_media_path=None, download_dir='./models/huggingface', load_format='auto', config_format=<ConfigFormat.AUTO: 'auto'>, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, max_model_len=None, guided_decoding_backend='outlines', distributed_executor_backend=None, worker_use_ray=False, pipeline_parallel_size=1, tensor_parallel_size=2, max_parallel_loading_workers=None, ray_workers_use_nsight=False, block_size=16, enable_prefix_caching=False, disable_sliding_window=False, use_v2_block_manager=False, num_lookahead_slots=0, seed=0, swap_space=4, cpu_offload_gb=0, gpu_memory_utilization=0.9, num_gpu_blocks_override=None, max_num_batched_tokens=None, max_num_seqs=256, max_logprobs=20, disable_log_stats=False, quantization=None, rope_scaling=None, rope_theta=None, hf_overrides=None, enforce_eager=False, max_seq_len_to_capture=8192, disable_custom_all_reduce=False, tokenizer_pool_size=0, tokenizer_pool_type='ray', tokenizer_pool_extra_config=None, limit_mm_per_prompt=None, mm_processor_kwargs=None, enable_lora=False, enable_lora_bias=False, max_loras=1, max_lora_rank=16, lora_extra_vocab_size=256, lora_dtype='auto', long_lora_scaling_factors=None, max_cpu_loras=None, fully_sharded_loras=False, enable_prompt_adapter=False, max_prompt_adapters=1, max_prompt_adapter_token=0, device='auto', num_scheduler_steps=1, multi_step_stream_outputs=True, scheduler_delay_factor=0.0, enable_chunked_prefill=None, speculative_model=None, speculative_model_quantization=None, num_speculative_tokens=None, speculative_disable_mqa_scorer=False, speculative_draft_tensor_parallel_size=None, speculative_max_model_len=None, speculative_disable_by_batch_size=None, ngram_prompt_lookup_max=None, ngram_prompt_lookup_min=None, spec_decoding_acceptance_method='rejection_sampler', typical_acceptance_sampler_posterior_threshold=None, typical_acceptance_sampler_posterior_alpha=None, disable_logprobs_during_spec_decoding=None, model_loader_extra_config=None, ignore_patterns=[], preemption_mode=None, served_model_name=None, qlora_adapter_name_or_path=None, otlp_traces_endpoint=None, collect_detailed_traces=None, disable_async_output_proc=False, scheduling_policy='fcfs', override_neuron_config=None, override_pooler_config=None, disable_log_requests=False, max_log_len=None, disable_fastapi_docs=False, enable_prompt_tokens_details=False, dispatch_function=<function serve at 0x7fc3b956dc60>)
```


### Download the model

To serve a new model vLLM will first download it to a local folder. By default it uses the `/home/.cache/huggingface` folder in Linux. The Llama3.3-70B model takes 132GB when all the safetensors are downloaded locally.

In AML, this folder is in the OS disk, which is only 120G in size and cannot hold a Llama3.3-70B model. So when running `vllm serve` we specify the `--download-dir` to a different folder in our compute instance's mounted file share, which is 100TiB. 

(The mount point of the file share is `/mnt/batch/tasks/shared/LS_root/mounts/clusters/<compute instance name>/code`, which seems to correspond to the top level folder that we see from the computer instance's Notebook app `/home/azureuser/cloudfiles/code`)

```bash
$ vllm serve unsloth/Llama-3.3-70B-Instruct \
$  --tensor-parallel-size 2 \
$  --download-dir "./models/huggingface"
```

The `unsloth/Llama-3.3-70B-Instruct` model will be downloaded as 30 safetensor shards, each of around 5 GiB in size.

### Load the model for serving

After the model is fully downloaded, vLLM will load all the safetensor shards into vRAM for serving. 

At first I ran into a weird error `zmq.error.ZMQError: Operation not supported`. To see the real issue, I disabled multi-processing by adding the parameter `--disable-frontend-multiprocessing` (reference: https://github.com/vllm-project/vllm/issues/10856).

```bash
$ vllm serve unsloth/Llama-3.3-70B-Instruct \
$   --tensor-parallel-size 2 \
$   --download-dir "./models/huggingface" \
$   --disable-frontend-multiprocessing
```    

Then I was able to see the real error: 
```bash
ValueError: The model's max seq len (131072) is larger than the maximum number of tokens that can be stored in KV cache (98384). Try increasing `gpu_memory_utilization` or decreasing `max_model_len` when initializing the engine.
```

The `max_model_len` parameter is by default set to the model's context length. In the case of Llama3.3-70B it is 128K, or `131072` bytes. The KV cache size was determined by vLLM and currently it cannot hold the max context length of tokens. 

I increased the `gpu_memory_utilization` (by default 0.9) to 0.95, still the KV cache was not enough to hold 128K context window. So I also decreased the `max_model_len`.

```bash
$ vllm serve unsloth/Llama-3.3-70B-Instruct \
$   --tensor-parallel-size 2 \
$   --download-dir "./models/huggingface" \
$   --disable-frontend-multiprocessing \
$   --gpu-memory-utilization 0.95 \
$   --max-model-len 128000
```

Finally, vLLM can serve the Llama3.3-70B using both GPUs.

```bash
INFO 12-09 04:38:22 model_runner.py:1077] Loading model weights took 65.7407 GB
(VllmWorkerProcess pid=79090) INFO 12-09 04:38:23 model_runner.py:1077] Loading model weights took 65.7407 GB
(VllmWorkerProcess pid=79090) INFO 12-09 04:38:25 worker.py:232] Memory profiling results: total_gpu_memory=93.02GiB initial_memory_usage=66.91GiB peak_torch_memory=65.84GiB memory_usage_post_profile=67.40GiB non_torch_memory=1.62GiB kv_cache_size=20.90GiB gpu_memory_utilization=0.95
INFO 12-09 04:38:25 worker.py:232] Memory profiling results: total_gpu_memory=93.02GiB initial_memory_usage=66.91GiB peak_torch_memory=66.95GiB memory_usage_post_profile=67.53GiB non_torch_memory=1.75GiB kv_cache_size=19.67GiB gpu_memory_utilization=0.95
INFO 12-09 04:38:26 distributed_gpu_executor.py:57] # GPU blocks: 8054, # CPU blocks: 1638
INFO 12-09 04:38:26 distributed_gpu_executor.py:61] Maximum concurrency for 128000 tokens per request: 1.01x
INFO 12-09 04:38:28 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-09 04:38:28 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=79090) INFO 12-09 04:38:28 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=79090) INFO 12-09 04:38:28 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=79090) INFO 12-09 04:38:42 custom_all_reduce.py:224] Registering 5635 cuda graph addresses
INFO 12-09 04:38:42 custom_all_reduce.py:224] Registering 5635 cuda graph addresses
(VllmWorkerProcess pid=79090) INFO 12-09 04:38:42 model_runner.py:1518] Graph capturing finished in 14 secs, took 2.28 GiB
INFO 12-09 04:38:42 model_runner.py:1518] Graph capturing finished in 14 secs, took 2.28 GiB
INFO 12-09 04:38:43 launcher.py:19] Available routes are:
INFO 12-09 04:38:43 launcher.py:27] Route: /openapi.json, Methods: HEAD, GET
INFO 12-09 04:38:43 launcher.py:27] Route: /docs, Methods: HEAD, GET
INFO 12-09 04:38:43 launcher.py:27] Route: /docs/oauth2-redirect, Methods: HEAD, GET
INFO 12-09 04:38:43 launcher.py:27] Route: /redoc, Methods: HEAD, GET
INFO 12-09 04:38:43 launcher.py:27] Route: /health, Methods: GET
INFO 12-09 04:38:43 launcher.py:27] Route: /tokenize, Methods: POST
INFO 12-09 04:38:43 launcher.py:27] Route: /detokenize, Methods: POST
INFO 12-09 04:38:43 launcher.py:27] Route: /v1/models, Methods: GET
INFO 12-09 04:38:43 launcher.py:27] Route: /version, Methods: GET
INFO 12-09 04:38:43 launcher.py:27] Route: /v1/chat/completions, Methods: POST
INFO 12-09 04:38:43 launcher.py:27] Route: /v1/completions, Methods: POST
INFO 12-09 04:38:43 launcher.py:27] Route: /v1/embeddings, Methods: POST
INFO:     Started server process [78834]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
```
